In [95]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import geopandas as gpd

In [96]:
wd = '/Users/kevin/Dropbox/legal_censorship/'

In [97]:
# Idea: For the subset of censored documents, we want to know whether "censorship" increases their chance of getting more government procurement.
# There could be 2 directions that we can explore 
    # --> whether "gaining" connection adds to the chance of the government's favor, 
    # --> or whether censorship helps the firm in terms of getting more procurements.
# To answer (1), we would need a dataset at firm (or maybe firm by year) level, with an indicator variable of connection (recent connection)
# To answer (2), we would need a dataset at firm-year, or even firm-month level, with an indicator variable of censorship at a certain year

In [98]:
df_censor = pd.read_csv(wd+'Data/master.csv')
df_censor_rescrape = pd.read_csv(wd+'Data/rescraping/output/rescraping_cases_100k-result.csv', header=None)
# 8.6 % of the master.csv is censored
df_censor = df_censor[df_censor.案号.isin(df_censor_rescrape[df_censor_rescrape[4]=='未查询到'][2])]
df_censor.原告loc = df_censor.原告loc.apply(literal_eval)
df_censor.被告loc = df_censor.被告loc.apply(literal_eval)
print(df_censor.columns)
df_rongzi = pd.read_csv(wd+'Data/company_data/tianyancha_database/td_gov_company_rongzi_basic.csv')
print(df_rongzi.columns)
# Match to full company list:
df_firmlist = pd.read_stata(wd+'Data/company_name_list/company_name_all_merged_tycbatch.dta')
df_firmlist_copy = df_firmlist.copy()
print(df_firmlist.columns)


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['文书ID', '案件名称', '案号', '审判程序', '裁判日期', '发布日期', '法院名称', '案件类型', '公开类型',
       '文书类型', '案由', 'docid', 'fname', 'county', 'pref', 'code', 'prefcode',
       '原告loc', '被告loc', 'type_原被告', '上诉', '原告pay', '被告pay', '类型_pay', '案件受理费',
       '受理费合计', '受理费减半', '受理费免收', '起诉', '申请', '立案', '受理', '撤诉', '开庭',
       'court_date', 'plantifflose', 'winner', 'loser', 'local_winner',
       'local_loser'],
      dtype='object')


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['id', 'company_id', 'disclosure_date', 'transaction_amount',
       'financing_rounds', 'valuation', 'proportion', 'investor',
       'news_sources', 'state', 'create_time', 'update_time', 'unified_code_x',
       'unified_code_y', 'ent_name', 'reg_capital', 'industry', 'start_date',
       'district', 'district_code', '_merge'],
      dtype='object')


/opt/anaconda3/lib/python3.7/site-packages/pandas/io/stata.py:1457: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


Index(['name', 'graphid', 'legalpersonname', 'businessterm', 'regstatus',
       'province', 'city', 'district', 'companyorgtype', 'historynames',
       'creditcode', 'taxcode', 'regnumber', 'orgnumber',
       'socialsecuritystaffnum', 'reglocation', 'start_year', 'start_month',
       'start_day', 'approve_year', 'approve_month', 'approve_day',
       'industry_category_code', 'regcapital_digital', 'actualcapital_digital',
       '_merge'],
      dtype='object')


In [99]:
df_firmlist.legalpersonname.isin(['']).mean()

0.1876050422504819

In [100]:
# df_firmlist = df_firmlist_copy.copy()
# First pass matching from legal data to firm data
df_subset = df_censor[['原告loc', '被告loc', '文书ID']]
df_temp = df_subset.explode('被告loc')
df_temp['被告'] = df_temp['被告loc'].apply(lambda x: x[0] if isinstance(x, list) else np.nan)
df_firmlist.drop_duplicates('name', inplace=True)
df_temp = pd.merge(df_temp.rename({'被告': 'name'}, axis=1), df_firmlist, on='name', validate='m:1', how='left')
print('Crude matching rate', df_temp.industry_category_code.isna().mean())
# Individuals (legal persons) related to firms can also be the key to the matching process
df_temp_matched = df_temp[~df_temp.industry_category_code.isna()]
df_temp = df_temp[df_temp.industry_category_code.isna()]
df_firmlist = df_firmlist[~df_firmlist.legalpersonname.isin([''])]
df_firmlist.drop_duplicates(subset='legalpersonname', inplace=True)
df_temp = df_temp[['原告loc', '被告loc', '文书ID', 'name']]
df_temp = pd.merge(df_temp.rename({'name': 'legalpersonname'}, axis=1), df_firmlist, on='legalpersonname', validate='m:1', how='left')
df_temp = pd.concat([df_temp, df_temp_matched])
print('Matching rate after name match', df_temp.industry_category_code.isna().mean())

Crude matching rate 0.6431083184412291
Matching rate after name match 0.4390384504525278


In [101]:
# Key columns: disclosure date, transaction_amount, financing_rounds, valuation
df_rongzi = df_rongzi[df_rongzi.financing_rounds == '定向增发']
def clean_amount(s):
    if s.endswith('万人民币') or s.endswith('万元人民币'):
        try:
            return float(s.replace('万人民币', '').replace('万元人民币', '')) * 1e4
        except:
            return np.nan
    elif s.endswith('亿人民币') or s.endswith('亿元人民币'):
        try:
            return float(s.replace('亿人民币', '').replace('亿元人民币', '')) * 1e8
        except:
            return np.nan
    else:
        return np.nan

df_rongzi['transaction_amount'] = df_rongzi['transaction_amount'].apply(clean_amount)
df_rongzi['transaction_amount'].fillna(df_rongzi.transaction_amount.mean(), inplace=True)
df_rongzi_copy = df_rongzi.copy()
df_rongzi.columns

Index(['id', 'company_id', 'disclosure_date', 'transaction_amount',
       'financing_rounds', 'valuation', 'proportion', 'investor',
       'news_sources', 'state', 'create_time', 'update_time', 'unified_code_x',
       'unified_code_y', 'ent_name', 'reg_capital', 'industry', 'start_date',
       'district', 'district_code', '_merge'],
      dtype='object')

In [102]:
df_rongzi = df_rongzi_copy.copy()
# The crude matching rate is 42% -- not ideal but ok -- need to come back to this step
print('matching rate', df_temp.industry_category_code.isna().mean())
# 826 out of 6191 firms gets at least one financing activity observed -- that's quite reasonable.
df_temp = df_temp[~df_temp.name.isna()]
print('# "censored" firms with >=1 financing activities', df_temp.name.isin(df_rongzi.ent_name).sum())

matching rate 0.4390384504525278
# "censored" firms with >=1 financing activities 100


In [108]:
df_rongzi = df_rongzi_copy.copy()
id_lst = df_rongzi.financing_rounds.value_counts().iloc[:15].index
def month_to_quarter(month):
    if month == '01' or month == '02' or month == '03':
        return 'Q1'
    if month == '04' or month == '05' or month == '06':
        return 'Q2'
    if month == '07' or month == '08' or month == '09':
        return 'Q3'
    if month == '10' or month == '11' or month == '12':
        return 'Q4'
df_rongzi['quarter'] = df_rongzi['disclosure_date'].apply(lambda x: x[:4]) + df_rongzi['disclosure_date'].apply(lambda x: month_to_quarter(x[5:7]))

# df_rongzi = df_rongzi[df_rongzi.ent_name.isin(df_temp.name)]
# df_rongzi = df_rongzi[df_rongzi.disclosure_date >= '2018']
for i in id_lst:
    df_rongzi[i] = df_rongzi.financing_rounds.isin([i])
df_rongzi['state'] = 1
df_rongzi['district_code'] = df_rongzi['district_code'].apply(str)
df_rongzi_panel = df_rongzi.groupby(['ent_name', 'quarter']).agg(
    num = ('state', 'sum'),
    industry = ('industry', 'first'),
    district_code = ('district_code', 'max'),
    amount = ('transaction_amount', 'sum')
)
df_rongzi_panel = df_rongzi_panel.reset_index().sort_values(['ent_name', 'quarter'])

for i in id_lst:
    df_rongzi_panel = pd.merge(df_rongzi_panel, df_rongzi.groupby(['ent_name', 'quarter'])[i].agg(sum).reset_index(), on=['ent_name', 'quarter'], how='left', validate='1:1')

In [109]:
# 599 rongzi incidences
# Transform it into a balanced panel
df_rongzi_panel.set_index('ent_name', inplace=True)
df_balanced = df_rongzi_panel.set_index('quarter', append=True).unstack().stack(dropna=False).reset_index()
df_balanced[['num']+[x for x in id_lst]] = df_balanced[['num']+[x for x in id_lst]].fillna(0)
df_balanced['post'] = df_balanced.quarter.apply(lambda x: x>'2021Q1')
df_subset = df_censor[['原告loc', '被告loc', '文书ID']]
df_temp = df_subset.explode('被告loc')
df_temp['被告'] = df_temp['被告loc'].apply(lambda x: x[0] if isinstance(x, list) else np.nan)
df_balanced['treat'] = df_balanced['ent_name'].isin(df_temp.被告.unique())

In [110]:
# The reasonable comparison group is not the full sample, but the firms that goes onto a litigation, but not receiving the favor of censorship.
df_uncensor = pd.read_csv('/Users/kevin/Dropbox/legal_censorship/Data/master_uncensor.csv')
df_uncensor.原告loc = df_uncensor.原告loc.apply(literal_eval)
df_uncensor.被告loc = df_uncensor.被告loc.apply(literal_eval)

df_temp2 = df_uncensor[['原告loc', '被告loc']]
df_temp2 = df_temp2.explode('被告loc')
df_temp2['被告'] = df_temp2['被告loc'].apply(lambda x: x[0] if isinstance(x, list) else np.nan)
df_balanced = df_balanced[(df_balanced.ent_name.isin(df_temp2.被告.unique())) | df_balanced['ent_name'].isin(df_temp.被告.unique())]

df_balanced.to_csv(wd+'Data/rongzi_firm_panel.csv', index=False)